In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Set a fixed random seed for reproducibility
np.random.seed(51)
tf.random.set_seed(51)

# load fashion mnist dataset
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train_full = x_train_full.astype("float32") / 255.0 - 0.5
x_test = x_test.astype("float32") / 255.0 - 0.5

x_train_full = np.reshape(x_train_full, (-1, 28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

num_classes = 10
y_train_full = keras.utils.to_categorical(y_train_full, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Shuffle the training data
indices = np.random.permutation(len(x_train_full))
x_train_full = x_train_full[indices]
y_train_full = y_train_full[indices]


# Split the training data into training and validation sets
val_size = 8000
x_val = x_train_full[:val_size]
y_val = y_train_full[:val_size]
x_train = x_train_full[val_size:]
y_train = y_train_full[val_size:]



# for train set: add mirrors of images from 0,2,4, and 6 categories e.g. T-shirt/top, Pullover, Coat, and Shirt
mask = np.isin(np.argmax(y_train, axis=1), [0, 2, 4, 6])
x_train_mirror = np.flip(x_train[mask], axis=2)
x_train_final = np.concatenate([x_train, x_train_mirror], axis=0)
y_train_final = np.concatenate([y_train, y_train[mask]], axis=0)


In [ ]:
# Model architecture
model = keras.Sequential([
    
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(rate=0.3),
    
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(rate=0.3),
    
    
    layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
    layers.Dropout(rate=0.3),
    
    layers.Flatten(),
    
    layers.Dense(128, activation="relu"),
    layers.Dropout(rate=0.3),
    
    layers.Dense(num_classes, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])




# Train the model 
batch_size = 512
epochs = 70
if tf.config.list_physical_devices('GPU'):
    print('Training on GPU...')
    with tf.device('/GPU:0'):
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(x_val, y_val))
else:
    print('Training on CPU...')
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                        validation_data=(x_val, y_val))


# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Training on GPU...
Epoch 1/70
102/102 [==============================] - 35s 50ms/step - loss: 0.9550 - accuracy: 0.6516 - val_loss: 0.5518 - val_accuracy: 0.7991
Epoch 2/70
102/102 [==============================] - 2s 24ms/step - loss: 0.5583 - accuracy: 0.7924 - val_loss: 0.4420 - val_accuracy: 0.8363
Epoch 3/70
102/102 [==============================] - 2s 24ms/step - loss: 0.4815 - accuracy: 0.8231 - val_loss: 0.3934 - val_accuracy: 0.8606
Epoch 4/70
102/102 [==============================] - 2s 23ms/step - loss: 0.4360 - accuracy: 0.8406 - val_loss: 0.3610 - val_accuracy: 0.8696
Epoch 5/70
102/102 [==============================] - 2s 24ms/step - loss: 0.4029 - accuracy: 0.8532 - val_loss: 0.3327 - val_accuracy: 0.8846
Epoch 6/70
102/102 [==============================] - 2s 24ms/step - loss: 0.3781 - accuracy: 0.8623 - val_loss: 0.3181 - val_accuracy: 0.8848
Epoch 7/70
102/102 [==============================] - 2s 24ms/step - loss: 0.3636 - accuracy: 0.8687 - val_loss: 0.3164 - 

Epoch 58/70
102/102 [==============================] - 3s 25ms/step - loss: 0.1811 - accuracy: 0.9321 - val_loss: 0.2107 - val_accuracy: 0.9259
Epoch 59/70
102/102 [==============================] - 3s 25ms/step - loss: 0.1786 - accuracy: 0.9328 - val_loss: 0.2075 - val_accuracy: 0.9252
Epoch 60/70
102/102 [==============================] - 3s 25ms/step - loss: 0.1754 - accuracy: 0.9327 - val_loss: 0.2160 - val_accuracy: 0.9240
Epoch 61/70
102/102 [==============================] - 3s 25ms/step - loss: 0.1767 - accuracy: 0.9335 - val_loss: 0.2088 - val_accuracy: 0.9269
Epoch 62/70
 80/102 [======================>.......] - ETA: 0s - loss: 0.1752 - accuracy: 0.9331

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# ВИСНОВОК: За рахунок використання згорткової мережі вдалося підвищити точність на 3.5% по відношенню до багатошарової мережі